### Q1a

In [1]:
#Creating DynamoDB table outside Lambda

import boto3
client = boto3.client('dynamodb')

dynamodb = boto3.resource('dynamodb')
table = dynamodb.create_table(
    TableName='psychrec',
    KeySchema=[
        {
            'AttributeName': 'username',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'username',
            'AttributeType': 'S'#string
        }
    ],
    ProvisionedThroughput={ #baseline concurrent capacity read and write
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }    
)


In [ ]:
# Code in Lambda

import time
import boto3
import json


dynamodb = boto3.resource('dynamodb')
client = boto3.client('dynamodb')#table created in jupyter?
table= dynamodb.Table('psychrec') 

s3 = boto3.resource('s3') 
s3.create_bucket(Bucket='psychlab-14070714')

time.sleep(10)

def lambda_handler(event, context):
    #saving to s3
    time = event["timestamp"]
    uid = event["user_id"]
    moo = event['mood']# string? int?
    key = time + '|' + uid
    eve = json.dumps(event)
    s3.Bucket('psychlab-14070714').put_object(Key=key,Body = eve) 
    
    #analyse sentiment
    txt = event["text"]
    comprehend = boto3.client('comprehend')
    response = comprehend.detect_sentiment(Text=txt,
                                       LanguageCode='en')
    sen = response['Sentiment']

    #saving to DB
    response = table.get_item(
        Key={
            'username': uid
        }
        )
    if 'Item' not in response.keys():  
        table.put_item(
           Item={
                'username': uid,
                'sentiment': sen,
                'mood': moo,
                'num_survey':1
            }
        )
    else:
        num = int(response['Item']['num_survey']) + 1
        table.update_item(
                Key={
                    'username': uid
                },
                UpdateExpression='SET num_survey = :val1, sentiment = :val2, mood = :val3',
                ExpressionAttributeValues={
                    ':val1': num,
                    ':val2': sen,
                    ':val3': moo,
                }
            )
        
    response = table.get_item(
        Key={
            'username': uid
        }
        )
    senti = response['Item']['sentiment']
    return senti



In [3]:
# Deleting S3 items and DynamoDB table outside Lambda
import boto3

client = boto3.client('dynamodb')
table= dynamodb.Table('psychrec') 
s3 = boto3.resource('s3') 

def cleanup(bucket_name):
    bucket = s3.Bucket(bucket_name)
    for item in bucket.objects.all():
        item.delete()
        
cleanup('psychlab-14070714')
table.delete()

{'TableDescription': {'TableName': 'psychrec',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 1,
   'WriteCapacityUnits': 1},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:087317734173:table/psychrec',
  'TableId': '90effbe1-2e51-41fe-a87c-013bd222164d'},
 'ResponseMetadata': {'RequestId': '1T3G1VT86ABSBN8VECB4GMNADJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 10 Nov 2021 19:34:07 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '318',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1T3G1VT86ABSBN8VECB4GMNADJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2273958093'},
  'RetryAttempts': 0}}

### Q1b

This solution mainly uses services DynamoDB and S3 buckets. Regarding DynamoDB, according to its documentation (https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Introduction.html), DynamoDB employs sufficient servers to handle throughputs and storage while maintaining consistency. Data is stored and replicated on availability zones in different places, thus further protecting data from damages of storage facilities. Especially with point-to-time recovery, any table can be restored within 35 days. Moreover, DynamoDB also ensures seamless scalability without downtime or degragation, suitable for study with a large amount of participants. In addition, as privacy is vital to psychological research, the encryption option within DynamoDB can protect the participants from privacy leakage.DynamoDB is also suitable for long-term storage of data, which can be used for future replication or further research purposes. Regarding S3, according to its documentation (https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html), S3 enables users to input and retrieve data of different time, without dealing with damaged or partial data, further ruling out data loss concerns. Similar to DynamoDB, S3 also replicates data across multiple servers to prevent data damage. Additionally, the eventual consistency of S3 buckets allows for time between the implementation and the realization of data changes, allowing users to make amendments when accidental changes are made. For example, if a bucket is accidentally deleted, it may still remain in the list for a while. 